# DROP TRIGGER statement

To delete a trigger from a table, you use the `DROP TRIGGER` statement with the following syntax:
```postgresql
DROP TRIGGER [IF EXISTS] trigger_name
ON table_name
[ CASCADE | RESTRICT ]; 
```

* **Use `IF EXISTS` to conditionally delete the trigger only if it exists.**
    * Deleting a non-existing trigger without specifying the `IF EXISTS` statement results in an error.
    * If you use `IF EXISTS` to delete a non-existing trigger, PostgreSQL issues a notice instead.
    * The `IF EXISTS` is optional.
* Specify the name of the table to which the trigger belongs. If the table belongs to a specific schema, you can use the schema-qualified name of the table, such as `schema_name.table_name`.
* Use the `CASCADE` option to drop objects that depend on the trigger automatically. Note that the `CASCADE` option will also delete objects that depend on objects that depend on the trigger.
* Use the `RESTRICT` option to refuse to drop the trigger if any objects depend on it. By default, the `DROP TRIGGER` statement uses `RESTRICT`.

Note that in SQL standard, trigger names are not local to tables, so the `DROP TRIGGER` statement does not have the table to which the trigger belongs:

```postgresql
DROP TRIGGER trigger_name; 
```

# DROP TRIGGER example

**First, create a function that validates the username of a staff.**

The username is not null, and its length must be at least 8.

```postgresql
CREATE FUNCTION check_staff_user()
    RETURNS TRIGGER
    LANGUAGE plpgsql; 
AS $$
BEGIN
    IF length(NEW.username) < 8 OR NEW.username IS NULL THEN
        RAISE EXCEPTION 'The username cannot be less than 8 characters';
    END IF;
    IF NEW.NAME IS NULL THEN
        RAISE EXCEPTION 'Username cannot be NULL';
    END IF;
    RETURN NEW;
END;
$$

```

**Second, create a new trigger on the `staff` table to check the username of a staff.**

This trigger will fire whenever you insert or update a row in the `staff` table.

```postgresql
CREATE TRIGGER username_check
    BEFORE INSERT OR UPDATE
ON staff
FOR EACH ROW
    EXECUTE PROCEDURE check_staff_user(); 
```

**Third, use the `DROP TRIGGER` statement to delete the `username_check trigger`**.

```postgresql
DROP TRIGGER username_check
ON staff; 
```